### Shared Word

In [97]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [98]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [99]:
lang_folder = "French"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language
lang_pair = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language
word_start = 0  # 0
word_end = 28  # 28 

In [100]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)

In [101]:
def remove_repetition(word_group):
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [102]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,de,16588988
1,je,16386475
2,pas,11547876
3,le,10592792
4,la,9939090
...,...,...
320426,perpetuel,5
320427,fifis,5
320428,fifteenth,5
320429,perouse,5


In [103]:
df_word = df_word_all.iloc[word_start:word_end,]
df_word

,word,frequency
0,de,16588988
1,je,16386475
2,pas,11547876
3,le,10592792
4,la,9939090
5,que,9858145
6,vous,8980559
7,tu,8776452
8,et,8039570
9,un,7421102


In [104]:
lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
lang_pair_list

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/French/French_And_Turkish (tur)_Etymologeek_Result_All.xlsx']

In [105]:
df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair

,dict_entry_main,turkish_word
0,perruque,peruk
1,palabre,palavra
2,parloir,salon
3,salon,salon
4,paraboler,parabol
...,...,...
825,équinoxe,ekinoks
826,érotique,erotik
827,éthique,etik
828,étiquette,etiket


In [106]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif"]

In [107]:
word_select = df_word["word"].values.tolist()

In [108]:
words = df_pair["dict_entry_main"].values.tolist()

In [109]:
word_select_set = set(word_select)
disable_word_set = set(disable_list)
words_set = set(words)

In [110]:
word_list = list(word_select_set.union(words_set.difference(disable_word_set)))

In [111]:
# 2 way  Not: 1 way is long and in Shared Word ReDe part 
data_kind_list = ["Two", "Three", "Four", "Five", "Sentence"]
for i in data_kind_list:
    sent_ngram_file = f"{i}" # Two, Three, Four, Five, Sentence
 
    if sent_ngram_file.lower() != "sentence":
        data_kind = f"{sent_ngram_file}gram"  # Twogram, Threegram, Fourgram, Fivegram
        folder_path = "N Gram"
    else:
        data_kind = "sentence"
        folder_path = "Sentence"

    df = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/{folder_path.capitalize()}/Merge/{data_kind.capitalize()}_Merge.csv")
    d_list  = df.iloc[:,0].values.tolist()

    resultlist = []
    manager = multiprocessing.Manager()
    resultlist = manager.list()
    
    def word_in_wordgroup(d_list):
        mergelist = []
        try:
            word = d_list.split()
        except:
            word = []
            #pass  disabled for non split value
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist.append(d_list)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup, d_list) # string_word liste

    result_list = list(resultlist)
    df_result = pd.DataFrame(result_list, columns=[0])  # add columns parameter for empty result
    df_result = df_result.rename(columns = {0: f"{data_kind.lower()}"})
    df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind.lower()}")
    df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
    df_merge_result.drop_duplicates(inplace=True)
    df_merge_result[f"{data_kind.lower()}"] = df_merge_result[f"{data_kind.lower()}"].apply(lambda x: remove_repetition(x))  # added remove_repetition 
    df_merge_result = df_merge_result[df_merge_result[f"{data_kind.lower()}"] != "repetitive_word_group"]  # remove_repetition part
    df_merge_result.reset_index(drop=True, inplace=True)

    df_merge_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_{data_kind.capitalize()}.xlsx", index=False)
    

#### Concat Result

In [112]:
df_word = pd.DataFrame(word_list, columns=["word"])
df_word

,word
0,mousquet
1,catégorie
2,terroriste
3,réactif
4,pantalon
...,...
833,primitif
834,paraboler
835,habillé
836,sodium


In [113]:
df_shared_twogram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram.xlsx")
df_shared_twogram = df_shared_twogram.loc[:,["twogram","frequency"]]
df_shared_twogram.rename(columns={"frequency":"freq_twogram"}, inplace=True)
df_shared_twogram

,twogram,freq_twogram
0,je ne,1251359
1,que tu,717094
2,que je,621189
3,ce que,529304
4,de la,527792
...,...,...
14415,ça surprise,2
14416,corvette en,2
14417,génération ce,2
14418,virtuose des,2


In [114]:
df_shared_threegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram.xlsx")
df_shared_threegram = df_shared_threegram.loc[:,["threegram","frequency"]]
df_shared_threegram.rename(columns={"frequency":"freq_threegram"}, inplace=True)
df_shared_threegram

,threegram,freq_threegram
0,ce que je,148993
1,ce que tu,125782
2,cest ce que,73726
3,ce que vous,71171
4,que tu as,65987
...,...,...
33144,le guichet et,2
33145,le guichet pour,2
33146,de ne as,2
33147,as rouge et,2


In [115]:
df_shared_fourgram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fourgram.xlsx")
df_shared_fourgram = df_shared_fourgram.loc[:,["fourgram","frequency"]]
df_shared_fourgram.rename(columns={"frequency":"freq_fourgram"}, inplace=True)
df_shared_fourgram

,fourgram,freq_fourgram
0,cest ce que je,27439
1,cest pour ça que,22014
2,ce que tu as,18277
3,cest ce que tu,14890
4,pas ce que je,12146
...,...,...
23783,jai le magasin je,2
23784,jai le maquillage qui,2
23785,boxe cest de la,2
23786,boxe cest pas que,2


In [116]:
df_shared_fivegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fivegram.xlsx")
df_shared_fivegram = df_shared_fivegram.loc[:,["fivegram","frequency"]]
df_shared_fivegram.rename(columns={"frequency":"freq_fivegram"}, inplace=True)
df_shared_fivegram

,fivegram,freq_fivegram
0,cest pour ça que je,8672
1,cest pour ça que tu,4661
2,tu as de la chance,2412
3,cest pas ce que je,2261
4,cest pour ça que vous,2240
...,...,...
7741,mais tu les tu les,2
7742,mais jai pas de champagne,2
7743,mais tu me les as,2
7744,mais jai pas de maîtresse,2


In [117]:
df_shared_sentence = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Sentence.xlsx")
df_shared_sentence = df_shared_sentence.loc[:,["sentence","frequency"]]
df_shared_sentence.rename(columns={"frequency":"freq_sentence"}, inplace=True)
df_shared_sentence

,sentence,freq_sentence
0,cest ça,58819
1,et vous,18817
2,cest qui,11327
3,cest super,10475
4,qui cest,10301
...,...,...
6227,cest pas un stage,2
6228,le flou cest vous docteur,2
6229,cest pas un trauma,2
6230,cest pas un terroriste,2


In [118]:
df_shared_all = pd.concat([df_word, df_shared_twogram, df_shared_threegram, df_shared_fourgram, df_shared_fivegram, df_shared_sentence], axis=1)
df_shared_all

,word,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,mousquet,je ne,1251359.0,ce que je,148993,cest ce que je,27439.0,cest pour ça que je,8672.0,cest ça,58819.0
1,catégorie,que tu,717094.0,ce que tu,125782,cest pour ça que,22014.0,cest pour ça que tu,4661.0,et vous,18817.0
2,terroriste,que je,621189.0,cest ce que,73726,ce que tu as,18277.0,tu as de la chance,2412.0,cest qui,11327.0
3,réactif,ce que,529304.0,ce que vous,71171,cest ce que tu,14890.0,cest pas ce que je,2261.0,cest super,10475.0
4,pantalon,de la,527792.0,que tu as,65987,pas ce que je,12146.0,cest pour ça que vous,2240.0,qui cest,10301.0
...,...,...,...,...,...,...,...,...,...,...,...
33144,NaN,NaN,NaN,le guichet et,2,NaN,NaN,NaN,NaN,NaN,NaN
33145,NaN,NaN,NaN,le guichet pour,2,NaN,NaN,NaN,NaN,NaN,NaN
33146,NaN,NaN,NaN,de ne as,2,NaN,NaN,NaN,NaN,NaN,NaN
33147,NaN,NaN,NaN,as rouge et,2,NaN,NaN,NaN,NaN,NaN,NaN


In [119]:
df_shared_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency.xlsx", index=False)

In [120]:
def word_in_twogram(df, list_column, target_column):

    '''word_in_twogram(df, "word", "twogram"):
       df is dataframe, list_column and target_column are 
       dataframe column string name. list_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{list_column}"].dropna():
        try:
            word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_twogram.insert(0,"word",i)
        df_word_result = pd.concat([df_word_result,word_in_twogram], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [121]:
df_word_order_twogram = word_in_twogram(df_shared_all, "word", "twogram")
df_word_order_threegram = word_in_twogram(df_shared_all, "word", "threegram") 
df_word_order_fourgram = word_in_twogram(df_shared_all, "word", "fourgram") 
df_word_order_fivegram = word_in_twogram(df_shared_all, "word", "fivegram")
df_word_order_sentence = word_in_twogram(df_shared_all, "word", "sentence") 

In [122]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_threegram = df_word_order_threegram.groupby(["word"])["threegram"].apply(", ".join).reset_index()
df_word_order_join_fourgram = df_word_order_fourgram.groupby(["word"])["fourgram"].apply(", ".join).reset_index()
df_word_order_join_fivegram = df_word_order_fivegram.groupby(["word"])["fivegram"].apply(", ".join).reset_index()
df_word_order_join_sentence = df_word_order_sentence.groupby(["word"])["sentence"].apply(", ".join).reset_index()

In [123]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram,df_word_order_join_fourgram,df_word_order_join_fivegram,df_word_order_join_sentence]

In [124]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all  

,word,twogram,threegram,fourgram,fivegram,sentence
0,abonné,"un abonné, est abonné, abonné de, pas abonné","un abonné de, il est abonné, pour un abonné",NaN,NaN,NaN
1,accent,"un accent, accent est, accent de, accent allem...","un accent allemand, cest un accent, un accent ...","tu as un accent, un accent allemand et, mais p...","accent en est un de, un accent américain pour ...","tu as un accent, un accent, jai un accent, un ..."
2,accord,"un accord, accord de, accord pour, en accord, ...","un accord de, cest un accord, un accord est, e...","un accord est un, accord est un accord, cest u...","un accord est un accord, cest un accord de pri...","un accord, un accord est un accord, cest un ac..."
3,acide,"un acide, et acide, cest acide, acide pour, ac...","sodium et acide, et un acide, pour un acide, u...","de sodium et acide, cest une réaction acide, e...",jai un acide mais je,"cest acide, un acide, et la douche acide"
4,acrobate,"un acrobate, une acrobate, acrobate et, acroba...","pour un acrobate, un acrobate qui, est un acro...",NaN,NaN,"une acrobate, un acrobate"
...,...,...,...,...,...,...
718,équinoxe,équinoxe de,NaN,NaN,NaN,NaN
719,érythrée,équateur érythrée,NaN,NaN,NaN,NaN
720,éthique,"pas éthique, une éthique, éthique de, éthique ...","jai une éthique, cest pas éthique, le code éth...","le code éthique et, code éthique et la, cest é...",le code éthique et la,"cest pas éthique, culture éthique, pas éthique..."
721,étiqueter,"étiqueter les, et étiqueter, les étiqueter, ét...","les étiqueter pour, et étiqueter les, de les é...",les étiqueter pour les,NaN,NaN


In [125]:
df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency.xlsx", index=False)

#### Copy Move And Delete

In [126]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*.xlsx")
output_file

['French_Turkish_Shared_Twogram.xlsx',
 'French_Turkish_Shared_Threegram.xlsx',
 'French_Turkish_Shared_Fourgram.xlsx',
 'French_Turkish_Shared_Fivegram.xlsx',
 'French_Turkish_Shared_Sentence.xlsx',
 'French_Turkish_Shared_Result_With_Frequency.xlsx',
 'French_Turkish_Shared_Join_Result_Without_Frequency.xlsx']

In [127]:
for k in output_file:
    source = k # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [128]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass